In [3]:
import numpy as np

DATA_PATH = "/public/liujiawei/huawei/ZHB/ADF-master/mnist/"
X_train = np.load(DATA_PATH+"mnist_train.npy")
Y_train = np.load(DATA_PATH+"mnist_train_label.npy")
X_test = np.load(DATA_PATH+"mnist_test.npy")
Y_test = np.load(DATA_PATH+"mnist_test_label.npy")

In [4]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [5]:
input_shape = 28*28

model = Sequential()
model.add(Dense(input_shape=(input_shape,), units=input_shape, activation='relu'))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
MODEL_PATH = "/public/liujiawei/huawei/ZHB/ADF-master/models/"
model = load_model(MODEL_PATH+"mnist_FC4.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
per_tmp = model.predict(X_test[0:1])
label_tmp = np.argmax(per_tmp)
print(per_tmp)
np.sum(per_tmp * per_tmp)

[[8.1880060e-19 1.1946783e-12 2.1346804e-13 7.7883793e-12 2.6276783e-11
  6.9444208e-15 2.8902736e-16 9.9999988e-01 3.6701772e-14 1.5277264e-07]]


0.99999976

In [8]:
Gini = []
index1 = 0
index2 = 0
for x_tmp in X_test:
    per_tmp = model.predict(x_tmp.reshape([-1,28*28]))
    label_tmp = np.argmax(per_tmp)
    if label_tmp == np.argmax(Y_test[index1]):
        index2 = index2 +1
    index1 = index1 + 1
    Gini_tmp = 1-np.sum(per_tmp*per_tmp)
    Gini.append(Gini_tmp)

In [9]:
print(index1, index1-index2)

10000 173


In [10]:
print(model.evaluate(X_test, Y_test, batch_size=128))

10000/10000 [==============================] - 0s 35us/sample - loss: 0.0813 - acc: 0.9827
[0.08125762678885899, 0.9827]


In [46]:
indexs = np.argsort(Gini)
indexs = indexs[::-1]

In [47]:
per_tmp = model.predict(X_test[6651].reshape([-1,28*28]))
print(1-np.sum(per_tmp*per_tmp), Gini[indexs[9999]])

0.8791627883911133 0.0


In [57]:
o_i = 0
for i in range(0, (index1-index2)):
    x_tmp = X_test[indexs[i]].reshape([-1,28*28])
    y_tmp = Y_test[indexs[i]]
    per_tmp = model.predict(x_tmp)
    if np.argmax(y_tmp)!=np.argmax(per_tmp):
        o_i = o_i+i
APFD = 1-o_i/(index1*(index1-index2))+1/(2*index1)
print(APFD)

0.9968193641618497


In [82]:
def softmax( f ):
    # instead: first shift the values of f so that the highest number is 0:
    f -= np.max(f) # f becomes [-666, -333, 0]
    return np.exp(f) / np.sum(np.exp(f))  # safe to do, gives the correct answer
    
from tensorflow.keras import backend as BE
import tensorflow as tf

get_activations = BE.function(inputs=model.inputs[0], outputs=model.layers[-2].output[:,:])

Gini_act = []
for x_tmp in X_test:
    x_act = get_activations(x_tmp.reshape([-1,28*28]))
    Gini_tmp = 1-np.sum(x_act*x_act)
    Gini_act.append(Gini_tmp)


In [83]:
indexs = np.argsort(Gini_act)
indexs = indexs[::-1]
o_i = 0
for i in range(0, (index1-index2)):
    x_tmp = X_test[indexs[i]].reshape([-1,28*28])
    y_tmp = Y_test[indexs[i]]
    per_tmp = model.predict(x_tmp)
    if np.argmax(y_tmp)!=np.argmax(per_tmp):
        o_i = o_i+i
APFD = 1-o_i/(index1*(index1-index2))+1/(2*index1)
print(APFD)

0.9976598265895954
